In [1]:
import warnings

warnings.filterwarnings('ignore')

import re
import json
import pandas as pd

from selenium import webdriver  # 导入库
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from fake_useragent import UserAgent
import pandas as pd
import time
import random

import requests
from lxml import etree

In [29]:
pd.set_option('max_row', 300)

In [2]:
def ini_browser():
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    #     options.add_argument('User-Agent={}'.format(UserAgent().random))
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    #    prefs = {"profile.managed_default_content_settings.images": 2}
    #    options.add_experimental_option("prefs", prefs)
    browser = webdriver.Chrome(chrome_options=options)
    browser.get('http://www.baidu.com')
    return browser

In [3]:
browser1 = ini_browser()
browser2 = ini_browser()

In [4]:
# base_url = 'http://data.bank.hexun.com/lccp/alllccp.aspx'
# browser1.get(base_url)

In [5]:
# i = 0
# while i<2:
#     browser1.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#     time.sleep(1)
#     i+=1
# time.sleep(5)

In [6]:
# tree1 = etree.HTML(browser1.page_source)  # 获取源码

In [7]:
p=1
all_info = []

while p<21:
    base_url = 'http://data.bank.hexun.com/lccp/AllLccp.aspx?col=fld_issenddate&tag=desc&orderMarks=&page={p}'.format(p=p)
    browser1.get(base_url)
    tree1 = etree.HTML(browser1.page_source) 
    
    t = 0
    while t<2:
        browser1.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        t+=1
    time.sleep(5)
    
    prod_baselink = 'http://data.bank.hexun.com/lccp/'
    i = 2

    while i <= 21:
        prod_name = tree1.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='mark']/table[@id='Table1']/tbody/tr[{i}]/td[@class='dr']/b/a".format(i=i))[0].text
        prod_sublink = tree1.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='mark']/table[@id='Table1']/tbody/tr[{i}]/td[@class='dr']/b/a//@href".format(i=i))[0]
        prod_link = prod_baselink + prod_sublink
        browser2.get(prod_link)
        tree2 = etree.HTML(browser2.page_source)

        prod_info = {}

        time.sleep(2)
        
        prod_name = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[@class='presell']/div[@class='fl prestxt']")[0].text
        prod_info['产品名称'] = prod_name
        print(prod_name)
        
        row = 1
        while row<=8:
            key1 = tree2.xpath("//div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[2]/table[@class='dzTable']/tbody/tr[{row}]/th[1]".format(row=row))[0].text.replace('：', '')
            val1 = tree2.xpath("//div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[2]/table[@class='dzTable']/tbody/tr[{row}]/td[1]".format(row=row))[0].text.replace('：', '')
            key2 = tree2.xpath("//div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[2]/table[@class='dzTable']/tbody/tr[{row}]/th[2]".format(row=row))[0].text.replace('：', '')
            val2 = tree2.xpath("//div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][1]/div[@class='Area443 fl']/div[2]/table[@class='dzTable']/tbody/tr[{row}]/td[2]".format(row=row))[0].text.replace('：', '')

            prod_info[key1] = val1
            prod_info[key2] = val2

            row += 1
            
        row2 = 1
        while row2<=3:
            key3 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/th[1]".format(row=row2))[0].text.replace('：', '')
            val3 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/td[1]".format(row=row2))[0].text.replace('：', '')
            key4 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/th[2]".format(row=row2))[0].text.replace('：', '')
            val4 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/td[2]".format(row=row2))[0].text.replace('：', '')

            prod_info[key3] = val3
            prod_info[key4] = val4

            row2 += 1
        
        row3 = 4
        while row3<=9:
            key5 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/th".format(row=row3))[0].text.replace('：', '')
            val5 = tree2.xpath("//div[@class='maincont']/div[@class='mainbox']/div[@class='mbox']/div[@class='dzMain']/div[@class='dzMain'][4]/div/table[@class='dzTable']/tbody/tr[{row}]/td".format(row=row3))[0].text.replace('：', '')
             
            prod_info[key5] = val5
            
            row3 += 1
            
        all_info.append(prod_info)

        i += 1
        
    p += 1

with open('/Users/Tianyu/Documents/OneConnect/Hackathon/Templates5/bank-manager-tool/backend/data/hx_dep_prd.json', 'a', encoding="utf-8") as f:
    json.dump(all_info, f, ensure_ascii=False, indent=4)
    f.write('\n')

搏弈BYAQKF280D
日日聚财(1401期)A19003
日日聚财(1401期)A19002
日日聚金1501期G26008
红棉理财C1082720A000010
紫气东来稳健1294期
翠竹同益3M对公10款(B)19期FGAB15044B
翠竹同益3M对公10款(A)19期FGAB15044A
紫气东来稳健1293期
翠竹同益1M54期FGAB15016A
翠竹同益3M对公09款(B)18期FGAB15042B
翠竹同益3M对公09款(A)18期FGAB15042A
成长2020年第74期
安稳2020年第42期
翠竹同益3M对公08款(B)18期FGAB15040B
翠竹同益3M对公08款(A)18期FGAB15040A
成长2020年第73期
合赢2020年第4期
成长2020年第71期
成长2020年第72期
金翼宝942号
安盈268号
安盈269号
安盈270号
安盈271号
安盈272号
安盈273号
安盈274号
安盈275号
安盈276号
安盈277号
安盈278号
安盈279号
安盈280号
金翼宝941号
翠竹5W周四尊享02款FSAB13004D
翠竹5W周四高端02款FSAB13004B
翠竹5W周四普通02款FSAB13004A
紫气东来稳健1292期
工银财富CQXT2020
金翼宝940号
成长2020年第70期
安稳2020年第41期
翠竹同益3M对公07款(B)18期FGAB15038B
翠竹同益3M对公07款(A)18期FGAB15038A
中银平稳智荟208496期
安稳2020年第40期
创赢计划青鑫共享2020年186期(封闭)
创赢计划青鑫共享2020年185期(封闭)
创赢计划青鑫共享2020年174期(封闭)
创赢计划青鑫共享2020年173期(封闭)
金翼宝939号
蒙银财鑫20057期
蒙银财鑫20056期
农银私行·安心得利·灵珑AD200210
优选恒利1号2020年第19期
金紫薇共赢43号封闭净值
金紫薇共赢42号封闭净值
和美保本型(第2007期)
金钥匙·安心得利·灵珑第209期
如意宝YB200089期
如意宝YB200090期
如意宝YB200087期
财坤CK200116期
鑫安利20107期
鑫安利20103期
鑫安利20104期
"芙蓉锦程·6个月"6号第2001投资周期
财富通稳赢

In [19]:
with open('/Users/Tianyu/Documents/OneConnect/Hackathon/Templates5/bank-manager-tool/backend/data/hx_dep_prd.json', encoding='utf-8') as data_file:
    data = json.loads(data_file.read())

In [20]:
df = pd.DataFrame(data)

In [30]:
df['发行银行'].value_counts()

民生银行    38
建设银行    30
广州农商    23
农业银行    21
交通银行    18
中国银行    17
兴业银行    16
晋商银行    16
工商银行    15
吉林银行    14
青岛银行    12
稠州银行    11
上海农商     9
广州银行     8
邮储银行     8
宁波银行     8
中信银行     7
内蒙古银     7
成都银行     6
上海银行     6
兰州银行     6
杭州联合     5
新韩银行     5
绍兴银行     5
盛京银行     5
杭州银行     4
湖北银行     4
泰安市商     4
乐山商行     4
南京银行     3
南商中国     3
河北银行     3
广东南粤     3
大连银行     3
招商银行     3
长沙银行     3
成都农商     3
广东华兴     3
齐商银行     3
紫金农商     2
嘉兴银行     2
海峡银行     2
恒丰银行     2
锦州银行     2
厦门银行     2
鄞州农村     2
天津银行     2
桂林银行     2
上饶银行     2
天津农商     2
攀枝花市     1
浙江民泰     1
汉口银行     1
德阳银行     1
西安银行     1
慈溪农村     1
龙江银行     1
日照银行     1
平安银行     1
浦发银行     1
潍坊银行     1
兴化农村     1
光大银行     1
广西北部     1
滨海农商     1
华融湘江     1
Name: 发行银行, dtype: int64

In [22]:
df.columns

Index(['产品名称', '发行银行', '委托币种', '投资类型', '委托币种起始金额', '预期收益率(%)', '起始金额递增单位',
       '到期收益率(%)', '同期存款利率比较', '是否保本', '发行起始日期', '可否质押', '发行终止日期', '委托管理期(月)',
       '收益起始日期', '付息周期(月)', '收益终止日期', '收益类型', '委托附属币种', '客户是否有权提前赎回',
       '附属币种起始金额', '银行是否有权提前终止', '附属币种起始金额递增单位', '适用地区', '产品管理费', '申购赎回规定',
       '投资对象', '收益率说明', '投资风险说明'],
      dtype='object')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   产品名称          400 non-null    object
 1   发行银行          400 non-null    object
 2   委托币种          400 non-null    object
 3   投资类型          400 non-null    object
 4   委托币种起始金额      400 non-null    object
 5   预期收益率(%)      400 non-null    object
 6   起始金额递增单位      400 non-null    object
 7   到期收益率(%)      400 non-null    object
 8   同期存款利率比较      400 non-null    object
 9   是否保本          400 non-null    object
 10  发行起始日期        400 non-null    object
 11  可否质押          400 non-null    object
 12  发行终止日期        400 non-null    object
 13  委托管理期(月)      400 non-null    object
 14  收益起始日期        400 non-null    object
 15  付息周期(月)       400 non-null    object
 16  收益终止日期        400 non-null    object
 17  收益类型          400 non-null    object
 18  委托附属币种        400 non-null    object
 19  客户是否有权提前

In [24]:
df['预期收益率(%)'].value_counts()

0       214
3.65     13
4.15     13
4.00     12
3.90      8
       ... 
1.60      1
4.13      1
7.90      1
8.30      1
3.00      1
Name: 预期收益率(%), Length: 68, dtype: int64

In [25]:
df['委托管理期(月)'].value_counts()

3.03     36
6.10     18
3.00     16
3.27     13
6.00     13
         ..
3.90      1
41.33     1
9.57      1
1.03      1
11.40     1
Name: 委托管理期(月), Length: 157, dtype: int64

In [26]:
df.shape

(400, 29)